In [76]:
%reload_ext autoreload
%autoreload 2

from IPython.display import display, Math

import numpy as np
from scipy.stats import ortho_group

from part3 import *

## Mathematical and Numerical Physics
### Numerical part 3
#### Kevin Vonk, s1706896, _Jan 2021_

## General part
### Question 1
#### a.

The summation term in eq. (1) of the assignment has the same form as an element-wise matrix multiplication, but the indices are off. The general form of matrix multiplication $\boldsymbol{C} = \boldsymbol{A}\boldsymbol{B}$ is (using the same indices as eq. (1) of the assignment),
\begin{align*}
    c_{jk} = \sum_{i=1}^n a_{ji}b_{ik}.
\end{align*}
From this we see that the summation index is the column index of the first operand and the row index of the second operand. We can achieve this form in eq. (1) of the assignment by transposing the first operand,
\begin{align*}
U_{ij} = U^T_{ji} = u'_{ji}.
\end{align*}
So,
\begin{align*}
    \sum_{i=1}^N u'_{ji}u_{ik} = c_{jk} = \delta_{jk}.
\end{align*}
From the delta we can deduce that when the indices $j$ and $k$ are equal, $c_{jk} = 1$. Otherwise, it is zero. These indices are only equal on the diagonal, so $\boldsymbol{C} = \boldsymbol{I}$. Changing from element-wise to general matrix products, we can write the final expression as,
\begin{align*}
    \boldsymbol{U}^T\boldsymbol{U} = \boldsymbol{I},
\end{align*}
which proves the equivalence.

# TODO: Prove UU^T = I and statement

#### b.
The results and code generating the results are found below.

In [77]:
dim = 3

U = ortho_group.rvs(dim)
I = np.identity(dim)

display(Math(r"\boldsymbol{U} = " + to_latex(U, 2)))

# Use np.allclose() to ignore IEEE 754 floating point rounding errors.
if np.allclose(U @ U.T, I) and np.allclose(U.T @ U, I):
    display(Math(r"\text{Python reports:}\quad\boldsymbol{U}\boldsymbol{U}^T = \boldsymbol{U}^T\boldsymbol{U} = \boldsymbol{I}."))
else:
    print("The statements i) and ii) do not hold.")


<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### c.
Firstly, lets prove symmetry. Starting of with the expression for $\boldsymbol{B}$,
\begin{align*}
    \boldsymbol{B} &= \boldsymbol{U}^T \boldsymbol{A} \boldsymbol{U} \\
    \boldsymbol{B}^T &= (\boldsymbol{U}^T \boldsymbol{A} \boldsymbol{U})^T = \boldsymbol{U}^T \boldsymbol{A}^T \boldsymbol{U} = \boldsymbol{U}^T \boldsymbol{A} \boldsymbol{U} \\
    &\rightarrow \boldsymbol{B}^T = \boldsymbol{B}.
\end{align*}

So, $\boldsymbol{B}$ is symmetric. Next, for the eigenvalues,
\begin{align*}
    \det(B) &= \det(U^T A U) = \det(U^T)\det(A)\det(U) \\
    &= \det(U^T)\det(U)\det(A) = \det(U^T U)\det(A) \\
    &= \det(I)\det(A) \\
    &= \det(A).
\end{align*}
And hence, by virtue of their determinants being identical, the eigenvalues $\lambda$ of $\boldsymbol{B}$ (determined by $\det(B - \lambda I) = 0$) will be identical too.

#### d.

In [78]:
A = np.diag([1, 2, 3])
B = U.T @ A @ U

valB, vecB = np.linalg.eig(B)

vecs = ",".join([to_latex(vecB[:, i].reshape((3,1)), 2, "pmatrix") for i in range(len(valB))])

display(Math(r"\text{Eigenvalues of}\; \boldsymbol{B} = " + to_latex(valB.reshape((3,1)), 0, "pmatrix")))
display(Math(r"\text{Eigenvectors of}\; \boldsymbol{B} = " + vecs))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

What we can see from the results is that the eigenvalues of $\boldsymbol{A}$ and $\boldsymbol{B}$ are identical. Additionally, the eigenvalues that have been found are the same values found on the diagonal of $\boldsymbol{A}$ (which makes sense, as all the off-diagonal components are zero). Lastly, the eigenvectors of $\boldsymbol{B}$ are rows of the $\boldsymbol{U}$ matrix. In this case the eigenvector corresponding to the lowest eigenvalue is the first row in $\boldsymbol{U}$, and the eigenvector corresponding to the highest eigenvalue is the last row, etc.

### Question 2
#### a.

In [79]:
rng = np.random.default_rng()
vec_start = rng.random(3)

eig_val, eig_vec, iter = eigen_power(B, vec_start, ret_cycles=True)

display(Math(r"\lambda_{max} = " + f"{eig_val:.4f}" + r"; \quad \text{Eigenvector}=" + to_latex(eig_vec.reshape(3,1)) + r"; \quad N_{iter} = " + f"{iter}"))

<IPython.core.display.Math object>

We can see that the power method algorithm has found the largest eigenvalue and corresponding eigenvalue. The amount of iterations required to reach $\epsilon < 10^{-8}$ is also given above. Note: I am deliberately not noting any numbers in this explaination text, since $\boldsymbol{U}$ (and thus $\boldsymbol{B}$ and its eigenvectors/eigenvalues) is based on random values which change in every run of the Notebook.

The eigenvalue found using this method is not 100% accurate. Its accuracy is determined by floating point rounding errors and the given accuracy. We know that the analytical maximum eigenvalue of $\boldsymbol{A}$ is $\lambda_{ana,max} = 3$. So, we can compare the analytical value $\lambda_{ana}$ to the value found using the Numpy library $\lambda_{Python}$ and to the value found using the power method $\lambda_{power}$.

In [80]:
l_ana = 3
diff_ana_python = np.abs(l_ana - np.max(valB))
diff_ana_power = np.abs(l_ana - eig_val)
diff_python_power = np.abs(np.max(valB) - eig_val)

display(Math(r"|\lambda_{ana} - \lambda_{Python}| = " + f"{diff_ana_python:.3e}"))
display(Math(r"|\lambda_{ana} - \lambda_{power}| = " + f"{diff_ana_power:.3e}"))
display(Math(r"|\lambda_{Python} - \lambda_{power}| = " + f"{diff_python_power:.3e}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

From these results we find that there is no difference between the analytical eigenvalue and the eigenvalue determined by Numpy. However, there is a difference between these two and our implemented power method, in the order of the provided accuracy $10^{-8}$. So, this difference seems reasonable.

#### b.

The same random starting vector as used in a) will be used again.

In [81]:
eig_val, eig_vec, iter = eigen_inv_power(B, vec_start, ret_cycles=True)

display(Math(r"\lambda_{max} = " + f"{eig_val:.4f}" + r"; \quad \text{Eigenvector}=" + to_latex(eig_vec.reshape(3,1)) + r"; \quad N_{iter} = " + f"{iter}"))

<IPython.core.display.Math object>

We can see from the results that we have computed the smallest eigenvalue and its corresponding eigenvector.

#### c.

We will again use the same random starting vector as we have done at a) and b). According to the lecture notes sec. 2.3.1, "If we choose $s$ close to $\lambda$ ... the best guess for $\lambda$ is then given by ...",

\begin{align*}
    \lambda \approx \frac{1}{S_n} + s.
\end{align*}

Since the last eigenvalue we are looking for is $\lambda = 2$, we will need to use a shift that is somewhat close to it in order to find this eigenvalue using the shifted inverse power method. Lets assume we are able to make a somewhat reasonable guess of the eigenvalue, so we pick $s = 1.7$. The results are found below.

In [82]:
eig_val, eig_vec, iter = eigen_shift_inv_power(B, vec_start, shift=1.7, ret_cycles=True)

display(Math(r"\lambda_{max} = " + f"{eig_val:.4f}" + r"; \quad \text{Eigenvector}=" + to_latex(eig_vec.reshape(3,1)) + r"; \quad N_{iter} = " + f"{iter}"))

<IPython.core.display.Math object>

We see that we have now obtained the last missing eigenvalue. Our value for the shift $s = 1.7$ seems to be a reasonable value since it brought us the correct eigenvalue we were looking for.

### Question 3
#### a.